In [2]:
import dagshub
import mlflow
from mlflow.client import MlflowClient
client = MlflowClient(tracking_uri='https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow',
                      registry_uri='https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow')
import pandas as pd
import numpy as np
import yaml
import os
import json
from pathlib import Path #type: ignore

from src.utils import load_yaml

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier, 
                              HistGradientBoostingClassifier, StackingClassifier, VotingClassifier)
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import hyperopt
from hyperopt.pyll.base import scope
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

os.chdir('f:\\iNeuron\\Projects\\Scania_Truck_Failures')

import mlflow.sklearn
import mlflow.xgboost
import mlflow.pyfunc

In [3]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Raj-Narayanan-B'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '8af4cc66be8aec751397fd525e47ae395fa67442'

In [4]:
mlflow.get_tracking_uri()

'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'

In [4]:
client.list_artifacts('c03a395f2448493aa7f2f5c4e066e0d3')

[<FileInfo: file_size=None, is_dir=True, path='optuna_Logistic_Regression'>]

In [ ]:
run_id = '7650b7097cb644d08b8fe61bb3065b15'
filter_string=f"tags.run_type ILIKE 'child'"
runs_df = mlflow.search_runs(experiment_ids=['60'])

In [ ]:
runs_df['artifact_uri'][0]

In [ ]:
best_run_id = mlflow.search_runs(experiment_ids=['60'],
                        filter_string=filter_string,
                        order_by = ['metrics.Cost'])[['run_id','artifact_uri','metrics.Cost']]['run_id'][0]
best_run_id

In [ ]:
mlflow.get_run(f'{best_run_id}')

In [ ]:
artifact_path_name = json.loads(mlflow.get_run(f'{best_run_id}').data.tags['mlflow.log-model.history'])[0]['artifact_path']

In [4]:
train_df = pd.read_csv("F:\iNeuron\Projects\scania_failures_2\\artifacts\data\processed\stage_2_processing\processed_train_data.csv")
test_df = pd.read_csv("F:\iNeuron\Projects\scania_failures_2\\artifacts\data\processed\stage_2_processing\processed_test_data.csv")

In [5]:
# x = df.drop(columns='class')
# y = df['class']
# # skf = StratifiedKFold()
# x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.25, random_state=8)
x_train,y_train = train_df.drop(columns='class'), train_df['class']
x_test,y_test = test_df.drop(columns='class'), test_df['class']

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

(2202, 170) (720, 170) (2202,) (720,)
class
0    1101
1    1101
Name: count, dtype: int64
class
0    360
1    360
Name: count, dtype: int64


In [ ]:
params = {'hidden_layer_sizes': (500, 300, 200, 150)}

In [ ]:
np.array(params['hidden_layer_sizes'])

In [ ]:
{"hidden_layer_sizes": np.array(params['hidden_layer_sizes'])}

In [ ]:
mlflow.models.infer_signature(x_train, y_test, {"hidden_layer_sizes": np.array(params['hidden_layer_sizes'])})

In [ ]:
params

In [ ]:
mlflow.log_params({"hidden_layer_sizes": [500, 300, 200, 150]})

In [ ]:
mlflow.set_tracking_uri()

In [ ]:
space = {'n_estimators': scope.int(hp.quniform('n_estimators', 100, 150, 1)),
    'criterion': hp.choice('criterion', ['log_loss', 'entropy', 'gini']),
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),
    'class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample'])}

mlflow.start_run()

mlflow.set_registry_uri
def objective(space):
    hp_rand_forest = RandomForestClassifier(**space)
    hp_rand_forest.fit(x_train,y_train)
    y_pred = hp_rand_forest.predict(x_test)
    tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
    cost = float((10*fp)+(500*fn))
    return cost

trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 1,
            trials= trials)
best_randf=space_eval(space,best)
best_randf

In [ ]:
import mlflow
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Create base classifiers
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

# Create stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=DecisionTreeClassifier()
)

# Log parameters
with mlflow.start_run():
    mlflow.log_params({
        'base_classifiers': base_classifiers,
        'final_estimator': 'DecisionTreeClassifier'
    })

    # Train and log other metrics as needed
    # ...

# Close the MLflow run
mlflow.end_run()


In [ ]:
eval('RandomForestClassifier()')

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

params = {'be1':['rf1','RandomForestClassifier()']}

mlflow.models.infer_signature(x_train, y_test, params)

In [ ]:
with mlflow.start_run(experiment_id = 75):
    mlflow.log_param(key = 'be1',
                     value = {'rf1','RandomForestClassifier()'})

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

# Create stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=DecisionTreeClassifier()
)

# # Log parameters
# with mlflow.start_run(experiment_id = 75):
#     mlflow.log_params({
#         'base_classifiers': eval('base_classifiers'),
#         'final_estimator': eval('DecisionTreeClassifier()')
#     })

In [ ]:
params = {
        'base_classifiers': eval('base_classifiers'),
        'final_estimator': eval('DecisionTreeClassifier()')
    }

In [ ]:
for key,value in params.items():
    params[key] = str(value)

params

In [ ]:
mlflow.models.infer_signature(x_train, y_test, new_params)

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]
str(eval('base_classifiers'))

In [ ]:
eval('base_classifiers')

In [11]:
new_params = client.get_run('ad5d40a9fc33487f8e616dde699ef7ec').data.params

[2024-01-24 22:35:09,693: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/runs/get?run_uuid=ad5d40a9fc33487f8e616dde699ef7ec&run_id=ad5d40a9fc33487f8e616dde699ef7ec]


In [12]:
new_params

{'cv': '5',
 'estimators': "[('KNN_Classifier', KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8))]",
 'final_estimator__algorithm': 'kd_tree',
 'final_estimator__leaf_size': '30',
 'final_estimator__metric': 'minkowski',
 'final_estimator__metric_params': 'None',
 'final_estimator__n_jobs': 'None',
 'final_estimator__n_neighbors': '8',
 'final_estimator__p': '2',
 'final_estimator__weights': 'uniform',
 'final_estimator': "KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8)",
 'n_jobs': '-1',
 'passthrough': 'False',
 'stack_method': 'predict',
 'verbose': '3',
 'KNN_Classifier': "KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8)",
 'KNN_Classifier__algorithm': 'kd_tree',
 'KNN_Classifier__leaf_size': '30',
 'KNN_Classifier__metric': 'minkowski',
 'KNN_Classifier__metric_params': 'None',
 'KNN_Classifier__n_jobs': 'None',
 'KNN_Classifier__n_neighbors': '8',
 'KNN_Classifier__p': '2',
 'KNN_Classifier__weights': 'uniform'}

In [ ]:
eval("[('KNN_Classifier', KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8))]")

In [13]:
for key,value in new_params.items():
                try:
                    new_params[key] = eval(value)
                except:
                    new_params[key] = value
                    if value == 'nan':
                        new_params[key] = np.nan
new_params                        

{'cv': 5,
 'estimators': [('KNN_Classifier',
   KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8))],
 'final_estimator__algorithm': 'kd_tree',
 'final_estimator__leaf_size': 30,
 'final_estimator__metric': 'minkowski',
 'final_estimator__metric_params': None,
 'final_estimator__n_jobs': None,
 'final_estimator__n_neighbors': 8,
 'final_estimator__p': 2,
 'final_estimator__weights': 'uniform',
 'final_estimator': KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8),
 'n_jobs': -1,
 'passthrough': False,
 'stack_method': 'predict',
 'verbose': 3,
 'KNN_Classifier': KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8),
 'KNN_Classifier__algorithm': 'kd_tree',
 'KNN_Classifier__leaf_size': 30,
 'KNN_Classifier__metric': 'minkowski',
 'KNN_Classifier__metric_params': None,
 'KNN_Classifier__n_jobs': None,
 'KNN_Classifier__n_neighbors': 8,
 'KNN_Classifier__p': 2,
 'KNN_Classifier__weights': 'uniform'}

In [14]:
processed_new_params = {key: value for key, value in new_params.items() if value is not None}

In [15]:
processed_new_params

{'cv': 5,
 'estimators': [('KNN_Classifier',
   KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8))],
 'final_estimator__algorithm': 'kd_tree',
 'final_estimator__leaf_size': 30,
 'final_estimator__metric': 'minkowski',
 'final_estimator__n_neighbors': 8,
 'final_estimator__p': 2,
 'final_estimator__weights': 'uniform',
 'final_estimator': KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8),
 'n_jobs': -1,
 'passthrough': False,
 'stack_method': 'predict',
 'verbose': 3,
 'KNN_Classifier': KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8),
 'KNN_Classifier__algorithm': 'kd_tree',
 'KNN_Classifier__leaf_size': 30,
 'KNN_Classifier__metric': 'minkowski',
 'KNN_Classifier__n_neighbors': 8,
 'KNN_Classifier__p': 2,
 'KNN_Classifier__weights': 'uniform'}

In [ ]:
stacking_clf_params = stacking_clf.get_params()
for key, value in stacking_clf_params.items():
    stacking_clf_params[key] = str(value)
stacking_clf_params

In [ ]:
exp_id_list = ['84','85','86']
mlflow.search_runs(experiment_ids = exp_id_list,
                            search_all_experiments = True,
                            filter_string = f"tags.run_type ilike 'parent'")

In [18]:
client.tracking_uri

'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'

In [5]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                  'Stacked_Classifier' : StackingClassifier,
                  'Voting_Classifier' : VotingClassifier
                  }

for key in models.keys():
    try:
        client.delete_registered_model(f"Challenger_{key}")
    except:
        pass

In [6]:
for i in range(269,272):
    try:
        mlflow.delete_experiment(i)
        print(i)
    except:
        pass

269
270
271


In [ ]:
mlflow.delete_experiment('136389952941582973')

In [ ]:
mlflow.delete_experiment('647179760550514699')

In [ ]:
mlflow.delete_experiment('297515542365458558')

In [ ]:
mlflow.get_tracking_uri()

In [10]:
for i in range(1,106):
    try:
        os.system(f'mlflow gc --experiment-ids {i}')
        print(i)
    except:
        pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105


In [19]:
[i() for i in models.values()]

[LogisticRegression(),
 SGDClassifier(),
 RandomForestClassifier(),
 AdaBoostClassifier(),
 GradientBoostingClassifier(),
 BaggingClassifier(),
 ExtraTreesClassifier(),
 HistGradientBoostingClassifier(),
 DecisionTreeClassifier(),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),

In [479]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  # 'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                #   'Stacked_Classifier' : StackingClassifier,
                #   'Voting_Classifier' : VotingClassifier
                  }
estimators = list(zip(models.keys(),[i() for i in models.values()]))
estimators

[('Logistic_Regression', LogisticRegression()),
 ('SGD_Classifier', SGDClassifier()),
 ('Random Forest', RandomForestClassifier()),
 ('Ada_Boost', AdaBoostClassifier()),
 ('Grad_Boost', GradientBoostingClassifier()),
 ('Bagging_Classifier', BaggingClassifier()),
 ('ExtraTreesClassifier', ExtraTreesClassifier()),
 ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
 ('Decision_Tree_Classifier', DecisionTreeClassifier()),
 ('KNN_Classifier', KNeighborsClassifier())]

In [21]:
stacking_clf = StackingClassifier(estimators = estimators,
                                  final_estimator = XGBClassifier(),
                                  stack_method = 'auto',
                                  passthrough=True)

In [22]:
stacking_clf.get_params()

{'cv': None,
 'estimators': [('Logistic_Regression', LogisticRegression()),
  ('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('ExtraTreesClassifier', ExtraTreesClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Decision_Tree_Classifier', DecisionTreeClassifier()),
  ('XGB_Classifier',
   XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 gamma=None, grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=None, max_bin=None,
                 max_cat_threshold=None, max_cat_to_onehot=None,


In [23]:
client.tracking_uri

'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'

In [24]:
client._registry_uri

'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'

In [26]:
stacked_clf_params = stacking_clf.get_params()

In [29]:
with mlflow.start_run(experiment_id = 141):
    mlflow.log_params({key: value for key, value in stacked_clf_params.items() if value is not None})

RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [30]:
{key: value for key, value in stacked_clf_params.items() if value is not None}

{'estimators': [('Logistic_Regression', LogisticRegression()),
  ('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('ExtraTreesClassifier', ExtraTreesClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Decision_Tree_Classifier', DecisionTreeClassifier()),
  ('XGB_Classifier',
   XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 gamma=None, grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=None, max_bin=None,
                 max_cat_threshold=None, max_cat_to_onehot=None,
             

In [257]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                #   'Stacked_Classifier' : StackingClassifier,
                #   'Voting_Classifier' : VotingClassifier
                  }
estimators = list(zip(models.keys(), [i() for i in models.values()]))#[value() if key != 'XGB_Classifier' else 'XGBClassifier()' for key,value in models.items()]))
estimators

[('Logistic_Regression', LogisticRegression()),
 ('SGD_Classifier', SGDClassifier()),
 ('Random Forest', RandomForestClassifier()),
 ('Ada_Boost', AdaBoostClassifier()),
 ('Grad_Boost', GradientBoostingClassifier()),
 ('Bagging_Classifier', BaggingClassifier()),
 ('ExtraTreesClassifier', ExtraTreesClassifier()),
 ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
 ('Decision_Tree_Classifier', DecisionTreeClassifier()),
 ('XGB_Classifier',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_dep

In [104]:
for i in range(len(estimators)):
    if estimators[i][0] == 'XGB_Classifier':
        xgb_params = {key: value for key, value in estimators[i][1].get_params().items() if value is not None}

xgb_params

{'objective': 'binary:logistic', 'enable_categorical': False, 'missing': nan}

In [158]:
xgb_params

{'objective': 'binary:logistic', 'enable_categorical': False, 'missing': nan}

In [92]:
new_str

'XGBClassifier(base_score=None, booster=None, callbacks=None, colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None, device=None, early_stopping_rounds=None, enable_categorical=False, eval_metric=None, feature_types=None, gamma=None, grow_policy=None, importance_type=None, interaction_constraints=None, learning_rate=None, max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None, max_depth=None, max_leaves=None, min_child_weight=None, missing=nan, monotone_constraints=None, multi_strategy=None, n_estimators=None, n_jobs=None, num_parallel_tree=None, random_state=None, ...)'

In [154]:
estimators

[('Logistic_Regression', LogisticRegression()),
 ('SGD_Classifier', SGDClassifier()),
 ('Random Forest', RandomForestClassifier()),
 ('Ada_Boost', AdaBoostClassifier()),
 ('Grad_Boost', GradientBoostingClassifier()),
 ('Bagging_Classifier', BaggingClassifier()),
 ('ExtraTreesClassifier', ExtraTreesClassifier()),
 ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
 ('Decision_Tree_Classifier', DecisionTreeClassifier()),
 ('KNN_Classifier', KNeighborsClassifier())]

In [60]:
for i in range(len(estimators)):
    if estimators[i][0] == 'XGB_Classifier':
        estimators[i][1].set_params(**xgb_params)

In [75]:
str(estimators)

"[('Logistic_Regression', LogisticRegression()), ('SGD_Classifier', SGDClassifier()), ('Random Forest', RandomForestClassifier()), ('Ada_Boost', AdaBoostClassifier()), ('Grad_Boost', GradientBoostingClassifier()), ('Bagging_Classifier', BaggingClassifier()), ('ExtraTreesClassifier', ExtraTreesClassifier()), ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()), ('Decision_Tree_Classifier', DecisionTreeClassifier()), ('KNN_Classifier', KNeighborsClassifier())]"

In [191]:
new_params = {'estimators' : estimators,
              'XGB_Classifier' : 'XGBClassifier()',
              'final_estimator' : 'XGBClassifier()',
              'stack_method' : 'auto',
              'passthrough' : 'True',
              'xgb_params': xgb_params}

In [192]:
new_params

{'estimators': [('Logistic_Regression', LogisticRegression()),
  ('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('ExtraTreesClassifier', ExtraTreesClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Decision_Tree_Classifier', DecisionTreeClassifier()),
  ('KNN_Classifier', KNeighborsClassifier())],
 'XGB_Classifier': 'XGBClassifier()',
 'final_estimator': 'XGBClassifier()',
 'stack_method': 'auto',
 'passthrough': 'True',
 'xgb_params': {'objective': 'binary:logistic',
  'enable_categorical': False,
  'missing': nan}}

In [193]:
with mlflow.start_run(experiment_id = 141):
    mlflow.log_params(new_params)
    # mlflow.log_param(key = 'XGBClassifier',
    #                  value = XGBClassifier())
    # mlflow.log_param(key = 'final_estimator',
    #                  value = XGBClassifier())

[2024-01-25 09:34:42,264: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/runs/create]


In [194]:
stacked_run = mlflow.get_run('aaaec579ec5f4a63bad4dd03a8c6f1ae')

In [236]:
import numpy as np
run_params = {}
for key,value in stacked_run.data.params.items():
    try:
        run_params[key] = eval(value)
    except:
        run_params[key] = value
        if value == 'nan':
                            run_params[key] = np.nan
run_params

{'estimators': [('Logistic_Regression', LogisticRegression()),
  ('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('ExtraTreesClassifier', ExtraTreesClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Decision_Tree_Classifier', DecisionTreeClassifier()),
  ('KNN_Classifier', KNeighborsClassifier())],
 'XGB_Classifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat

In [237]:
run_params['xgb_params'] = run_params['xgb_params'].replace("nan",'None')
xgb_params_ = eval(run_params['xgb_params'])
xgb_params['missing'] = np.nan
xgb_params

{'objective': 'binary:logistic', 'enable_categorical': False, 'missing': nan}

In [239]:
run_params['estimators'].append(('XGBClassifier',run_params['XGB_Classifier'].set_params(**xgb_params)))

In [240]:
run_params

{'estimators': [('Logistic_Regression', LogisticRegression()),
  ('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('ExtraTreesClassifier', ExtraTreesClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Decision_Tree_Classifier', DecisionTreeClassifier()),
  ('KNN_Classifier', KNeighborsClassifier()),
  ('XGBClassifier',
   XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 gamma=None, grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=None, max_bin=None,
                 max_cat_threshol

In [354]:
estimators

[('Logistic_Regression', LogisticRegression()),
 ('SGD_Classifier', SGDClassifier()),
 ('Random Forest', RandomForestClassifier()),
 ('Ada_Boost', AdaBoostClassifier()),
 ('Grad_Boost', GradientBoostingClassifier()),
 ('Bagging_Classifier', BaggingClassifier()),
 ('ExtraTreesClassifier', ExtraTreesClassifier()),
 ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
 ('Decision_Tree_Classifier', DecisionTreeClassifier()),
 ('XGB_Classifier',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_dep

In [355]:
s_clf = StackingClassifier(estimators=estimators,
                   final_estimator=XGBClassifier(),
                   cv=5,
                   stack_method = 'auto',
                   passthrough=True)

# estimators,
# final_estimator,
# cv,
# stack_method,
# passthrough
s_clf.get_params()

{'cv': 5,
 'estimators': [('Logistic_Regression', LogisticRegression()),
  ('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('ExtraTreesClassifier', ExtraTreesClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Decision_Tree_Classifier', DecisionTreeClassifier()),
  ('XGB_Classifier',
   XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 gamma=None, grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=None, max_bin=None,
                 max_cat_threshold=None, max_cat_to_onehot=None,
   

In [338]:
s_clf.get_params()['estimators']
s_clf.get_params()['final_estimator']
s_clf.get_params()['cv']
s_clf.get_params()['stack_method']
s_clf.get_params()['passthrough']

True

In [371]:
s_clf.get_params()['estimators']

[('Logistic_Regression', LogisticRegression()),
 ('SGD_Classifier', SGDClassifier()),
 ('Random Forest', RandomForestClassifier()),
 ('Ada_Boost', AdaBoostClassifier()),
 ('Grad_Boost', GradientBoostingClassifier()),
 ('Bagging_Classifier', BaggingClassifier()),
 ('ExtraTreesClassifier', ExtraTreesClassifier()),
 ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
 ('Decision_Tree_Classifier', DecisionTreeClassifier()),
 ('XGB_Classifier',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_dep

In [385]:
f"{s_clf.get_params()['final_estimator'].__class__.__name__}_params"

'XGBClassifier_params'

In [345]:
flag = 0
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        flag =1
if flag == 1:
    clf_list = []
    for i in range(len(s_clf.get_params()['estimators'])):
        if s_clf.get_params()['estimators'][i][0] != "XGB_Classifier":
            clf_list.append(s_clf.get_params()['estimators'][i])
    estimators_params = {}
    for i in range(len(s_clf.get_params()['estimators'])):
        estimators_params[s_clf.get_params()['estimators'][i][0]] = s_clf.get_params()['estimators'][i][1].get_params()
        if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
            xgb_params = s_clf.get_params()['estimators'][i][1].get_params()
    xgb_params = {key:value for key,value in xgb_params.items() if value is not None}
    
    if s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier':
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : 'XGBClassifier()',
                'cv' : 8,#s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough'],
                'xgb_params' : xgb_params}
    else:
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough'],
                'xgb_params' : xgb_params}

    mlflow.log_params(s_clf_params)
    mlflow.models.infer_signature(model_input=x_train,
                              model_output = s_clf.predict(x_train))
    mlflow.sklearn.log_model()
    

[2024-01-25 11:29:26,147: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/runs/create]


In [357]:
s_clf.fit(x_train,y_train)

StackingClassifier(cv=5,
                   estimators=[('Logistic_Regression', LogisticRegression()),
                               ('SGD_Classifier', SGDClassifier()),
                               ('Random Forest', RandomForestClassifier()),
                               ('Ada_Boost', AdaBoostClassifier()),
                               ('Grad_Boost', GradientBoostingClassifier()),
                               ('Bagging_Classifier', BaggingClassifier()),
                               ('ExtraTreesClassifier', ExtraTreesClassifier()),
                               ('Hist_Grad_Boost_Clas...
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=None, ...),
                   passthrough=True)

In [358]:
s_clf_params_ = s_clf.get_params()
for key, value in s_clf_params_.items():
    s_clf_params_[key] = str(value)
signature = mlflow.models.infer_signature(model_input = x_train,
                                          model_output = s_clf.predict(x_train),
                                          params = s_clf_params_)

In [353]:
signature

inputs: 
  ['aa_000': double, 'ab_000': double, 'ac_000': double, 'ad_000': double, 'ae_000': double, 'af_000': double, 'ag_000': double, 'ag_001': double, 'ag_002': double, 'ag_003': double, 'ag_004': double, 'ag_005': double, 'ag_006': double, 'ag_007': double, 'ag_008': double, 'ag_009': double, 'ah_000': double, 'ai_000': double, 'aj_000': double, 'ak_000': double, 'al_000': double, 'am_0': double, 'an_000': double, 'ao_000': double, 'ap_000': double, 'aq_000': double, 'ar_000': double, 'as_000': double, 'at_000': double, 'au_000': double, 'av_000': double, 'ax_000': double, 'ay_000': double, 'ay_001': double, 'ay_002': double, 'ay_003': double, 'ay_004': double, 'ay_005': double, 'ay_006': double, 'ay_007': double, 'ay_008': double, 'ay_009': double, 'az_000': double, 'az_001': double, 'az_002': double, 'az_003': double, 'az_004': double, 'az_005': double, 'az_006': double, 'az_007': double, 'az_008': double, 'az_009': double, 'ba_000': double, 'ba_001': double, 'ba_002': double, 

In [335]:
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        xgb_params = s_clf.get_params()['estimators'][i][1].get_params()
xgb_params = {key:value for key,value in xgb_params.items() if value is not None}
xgb_params

{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


{'objective': 'binary:logistic', 'enable_categorical': False, 'missing': nan}

In [272]:
xgb = XGBClassifier()

In [276]:
xgb_params = {key:value for key,value in xgb.get_params().items() if value is not None}
xgb_params


{'objective': 'binary:logistic', 'enable_categorical': False, 'missing': nan}

In [336]:
s_clf_params

{'estimators': [('Logistic_Regression', LogisticRegression()),
  ('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('ExtraTreesClassifier', ExtraTreesClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Decision_Tree_Classifier', DecisionTreeClassifier()),
  ('KNN_Classifier', KNeighborsClassifier())],
 'final_estimator': 'XGBClassifier()',
 'cv': 5,
 'stack_method': 'auto',
 'passthrough': True,
 'xgb_params': {'objective': 'binary:logistic',
  'enable_categorical': False,
  'missing': nan}}

In [285]:
s_clf_params = {'estimators' : clf_list,
          'final_estimator' : 'XGBClassifier()',
          'cv' : s_clf.get_params()['cv'],
          'stack_method' : s_clf.get_params()['stack_method'],
          'passthrough' : s_clf.get_params()['passthrough'],
          'xgb_params' : xgb_params}

s_clf_params

{'estimators': [('Logistic_Regression', LogisticRegression()),
  ('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('ExtraTreesClassifier', ExtraTreesClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Decision_Tree_Classifier', DecisionTreeClassifier()),
  ('KNN_Classifier', KNeighborsClassifier())],
 'final_estimator': 'XGBClassifier()',
 'cv': 5,
 'stack_method': 'auto',
 'passthrough': True,
 'xgb_params': {'objective': 'binary:logistic',
  'enable_categorical': False,
  'missing': nan}}

In [459]:
trial_params = {'estimators': [('SGD_Classifier', SGDClassifier(loss='log_loss')), 
                ('Random Forest', RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy', max_features=None, n_estimators=139)), 
                ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier(max_iter=575)), 
                ('Bagging_Classifier', BaggingClassifier(n_estimators=59)), 
                ('Ada_Boost', AdaBoostClassifier(algorithm='SAMME', n_estimators=133)), 
                ('Grad_Boost', GradientBoostingClassifier(criterion='squared_error', loss='exponential', max_features='log2', n_estimators=143))], 
 'final_estimator': 'XGBClassifier()', 
 'cv': 5, 
 'stack_method': 'auto', 
 'passthrough': False, 
 'xgb_params': {'objective': 'binary:logistic', 
                'booster': 'dart', 
                'enable_categorical': False, 
                'learning_rate': 6.5697687477009445, 
                'missing': np.nan, 
                'n_estimators': 164, 
                'tree_method': 'hist'}
}

In [391]:
trial_params['estimators'].append(('XGB_Classifier',XGBClassifier()))
trial_params

{'estimators': [('SGD_Classifier', SGDClassifier(loss='log_loss')),
  ('Random Forest',
   RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                          max_features=None, n_estimators=139)),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier(max_iter=575)),
  ('Bagging_Classifier', BaggingClassifier(n_estimators=59)),
  ('Ada_Boost', AdaBoostClassifier(algorithm='SAMME', n_estimators=133)),
  ('Grad_Boost',
   GradientBoostingClassifier(criterion='squared_error', loss='exponential',
                              max_features='log2', n_estimators=143)),
  ('XGB_Classifier',
   XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 gamma=None, grow_policy=None, importance_type=None,
    

In [460]:
trial_params['estimators']

[('SGD_Classifier', SGDClassifier(loss='log_loss')),
 ('Random Forest',
  RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                         max_features=None, n_estimators=139)),
 ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier(max_iter=575)),
 ('Bagging_Classifier', BaggingClassifier(n_estimators=59)),
 ('Ada_Boost', AdaBoostClassifier(algorithm='SAMME', n_estimators=133)),
 ('Grad_Boost',
  GradientBoostingClassifier(criterion='squared_error', loss='exponential',
                             max_features='log2', n_estimators=143))]

In [461]:
s_clf.get_params()['final_estimator']

RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       max_features=None, n_estimators=139)

In [468]:
clf_list = []
for i in range(len(trial_params['estimators'])):
    clf_list.append((trial_params['estimators'][i][0],trial_params['estimators'][i][1].__class__()))

estimators_params = {}
for i in range(len(trial_params['estimators'])):
        estimators_params[trial_params['estimators'][i][0]] = trial_params['estimators'][i][1].get_params()
for i in estimators_params:
        estimators_params[i] = {key:value for key,value in estimators_params[i].items() if value is not None}
s_clf_params_trial_2 = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough']}
for key,value in estimators_params.items():
        s_clf_params_trial_2[f"{key}_params"] = value
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params_trial_2)

[2024-01-25 14:36:11,745: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/runs/create]


In [463]:
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params_trial)

[2024-01-25 14:28:57,497: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/runs/create]


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [453]:
s_clf = StackingClassifier(estimators = trial_params['estimators'],
                           final_estimator=RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                         max_features=None, n_estimators=139),
                           cv=5,
                           stack_method = 'auto',
                           passthrough=True)

In [454]:
s_clf

StackingClassifier(cv=5,
                   estimators=[('SGD_Classifier',
                                SGDClassifier(loss='log_loss')),
                               ('Random Forest',
                                RandomForestClassifier(class_weight='balanced_subsample',
                                                       criterion='entropy',
                                                       max_features=None,
                                                       n_estimators=139)),
                               ('Hist_Grad_Boost_Classifier',
                                HistGradientBoostingClassifier(max_iter=575)),
                               ('Bagging_Classifier',
                                BaggingClassifier(n_estimators=59)),...
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))],
                   final_estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                                          criterion='entropy',
                                                          max_features=None,
                                                          n_estimators=139),
                   passthrough=True)

In [423]:
s_clf.get_params()['estimators'][5][1].__class__()

GradientBoostingClassifier()

In [458]:
flag = 0
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        flag =1
if flag == 1:
    clf_list = []
    for i in range(len(s_clf.get_params()['estimators'])):
        if s_clf.get_params()['estimators'][i][0] != "XGB_Classifier":
            clf_list.append((s_clf.get_params()['estimators'][i][0],s_clf.get_params()['estimators'][i][1].__class__()))
    estimators_params = {}
    for i in range(len(s_clf.get_params()['estimators'])):
        estimators_params[s_clf.get_params()['estimators'][i][0]] = s_clf.get_params()['estimators'][i][1].get_params()

    for i in estimators_params:
        estimators_params[i] = {key:value for key,value in estimators_params[i].items() if value is not None}
    
    if s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier':
        
            s_clf_params = {'estimators' : clf_list,
                    'final_estimator' : 'XGBClassifier()',
                    'cv' : s_clf.get_params()['cv'],
                    'stack_method' : s_clf.get_params()['stack_method'],
                    'passthrough' : s_clf.get_params()['passthrough']}
            for key,value in estimators_params.items():
                s_clf_params[f"{key}_params"] = value

    else:
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough']}
        for key,value in estimators_params.items():
                s_clf_params[f"{key}_params"] = value

with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params)
#     # mlflow.models.infer_signature(model_input=x_train,
#     #                           model_output = s_clf.predict(x_train))
#     # mlflow.sklearn.log_model()
    

[2024-01-25 14:16:07,252: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/runs/create]


In [456]:
s_clf_params

{'estimators': [('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier())],
 'final_estimator': RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                        max_features=None, n_estimators=139),
 'cv': 5,
 'stack_method': 'auto',
 'passthrough': True,
 'SGD_Classifier_params': {'alpha': 0.0001,
  'average': False,
  'early_stopping': False,
  'epsilon': 0.1,
  'eta0': 0.0,
  'fit_intercept': True,
  'l1_ratio': 0.15,
  'learning_rate': 'optimal',
  'loss': 'log_loss',
  'max_iter': 1000,
  'n_iter_no_change': 5,
  'penalty': 'l2',
  'power_t': 0.5,
  'shuffle': True,
  'tol': 0.001,
  'validation_fraction': 0.1,
  'verbose': 0,
  'warm_start': False},
 'Random Forest_params': {'bootstrap': True,
  'ccp_alpha': 0.0,
  'cla

In [457]:
s_clf.get_params()['final_estimator']

RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       max_features=None, n_estimators=139)

In [450]:
s_clf_params

{'estimators': [('SGD_Classifier', SGDClassifier()),
  ('Random Forest', RandomForestClassifier()),
  ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()),
  ('Bagging_Classifier', BaggingClassifier()),
  ('Ada_Boost', AdaBoostClassifier()),
  ('Grad_Boost', GradientBoostingClassifier())],
 'final_estimator': 'XGBClassifier()',
 'cv': 5,
 'stack_method': 'auto',
 'passthrough': True,
 'SGD_Classifier_params': {'alpha': 0.0001,
  'average': False,
  'early_stopping': False,
  'epsilon': 0.1,
  'eta0': 0.0,
  'fit_intercept': True,
  'l1_ratio': 0.15,
  'learning_rate': 'optimal',
  'loss': 'log_loss',
  'max_iter': 1000,
  'n_iter_no_change': 5,
  'penalty': 'l2',
  'power_t': 0.5,
  'shuffle': True,
  'tol': 0.001,
  'validation_fraction': 0.1,
  'verbose': 0,
  'warm_start': False},
 'Random Forest_params': {'bootstrap': True,
  'ccp_alpha': 0.0,
  'class_weight': 'balanced_subsample',
  'criterion': 'entropy',
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 1,
  'mi

In [445]:
s_clf.get_params()['estimators'][0][0]

'SGD_Classifier'

In [446]:
estimators_params

{'SGD_Classifier': {'alpha': 0.0001,
  'average': False,
  'early_stopping': False,
  'epsilon': 0.1,
  'eta0': 0.0,
  'fit_intercept': True,
  'l1_ratio': 0.15,
  'learning_rate': 'optimal',
  'loss': 'log_loss',
  'max_iter': 1000,
  'n_iter_no_change': 5,
  'penalty': 'l2',
  'power_t': 0.5,
  'shuffle': True,
  'tol': 0.001,
  'validation_fraction': 0.1,
  'verbose': 0,
  'warm_start': False},
 'Random Forest': {'bootstrap': True,
  'ccp_alpha': 0.0,
  'class_weight': 'balanced_subsample',
  'criterion': 'entropy',
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 139,
  'oob_score': False,
  'verbose': 0,
  'warm_start': False},
 'Hist_Grad_Boost_Classifier': {'early_stopping': 'auto',
  'l2_regularization': 0.0,
  'learning_rate': 0.1,
  'loss': 'log_loss',
  'max_bins': 255,
  'max_iter': 575,
  'max_leaf_nodes': 31,
  'min_samples_leaf': 20,
  'n_iter_no_change': 10,
  'scoring': 'loss',
  'to

In [415]:
s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier'

True

In [437]:
name_of_dictionary = "my_dict"
globals()[name_of_dictionary] = {'hi':1}

In [436]:
name_of_dictionary

'my_dict'

In [405]:
estimators_params['XGB_Classifier'].items()

dict_items([('objective', 'binary:logistic'), ('base_score', None), ('booster', None), ('callbacks', None), ('colsample_bylevel', None), ('colsample_bynode', None), ('colsample_bytree', None), ('device', None), ('early_stopping_rounds', None), ('enable_categorical', False), ('eval_metric', None), ('feature_types', None), ('gamma', None), ('grow_policy', None), ('importance_type', None), ('interaction_constraints', None), ('learning_rate', None), ('max_bin', None), ('max_cat_threshold', None), ('max_cat_to_onehot', None), ('max_delta_step', None), ('max_depth', None), ('max_leaves', None), ('min_child_weight', None), ('missing', nan), ('monotone_constraints', None), ('multi_strategy', None), ('n_estimators', None), ('n_jobs', None), ('num_parallel_tree', None), ('random_state', None), ('reg_alpha', None), ('reg_lambda', None), ('sampling_method', None), ('scale_pos_weight', None), ('subsample', None), ('tree_method', None), ('validate_parameters', None), ('verbosity', None)])

In [367]:
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(trial_params)

In [315]:
mlflow.models.infer_signature(model_input=x_train,
                              model_output = s_clf.predict(x_train))

inputs: 
  ['aa_000': double, 'ab_000': double, 'ac_000': double, 'ad_000': double, 'ae_000': double, 'af_000': double, 'ag_000': double, 'ag_001': double, 'ag_002': double, 'ag_003': double, 'ag_004': double, 'ag_005': double, 'ag_006': double, 'ag_007': double, 'ag_008': double, 'ag_009': double, 'ah_000': double, 'ai_000': double, 'aj_000': double, 'ak_000': double, 'al_000': double, 'am_0': double, 'an_000': double, 'ao_000': double, 'ap_000': double, 'aq_000': double, 'ar_000': double, 'as_000': double, 'at_000': double, 'au_000': double, 'av_000': double, 'ax_000': double, 'ay_000': double, 'ay_001': double, 'ay_002': double, 'ay_003': double, 'ay_004': double, 'ay_005': double, 'ay_006': double, 'ay_007': double, 'ay_008': double, 'ay_009': double, 'az_000': double, 'az_001': double, 'az_002': double, 'az_003': double, 'az_004': double, 'az_005': double, 'az_006': double, 'az_007': double, 'az_008': double, 'az_009': double, 'ba_000': double, 'ba_001': double, 'ba_002': double, 

In [ ]:
v_clf = VotingClassifier()

In [474]:
s_clf_run = client.get_run('3f360d91251446b488d0255f44ea1658')

In [476]:
best_model = mlflow.sklearn.load_model('mlflow-artifacts:/11856c6b7da848c38f85aa3ece46ad6c/3f360d91251446b488d0255f44ea1658/artifacts/optuna_Stacked_Classifier')

In [477]:
best_model.get_params()

{'cv': 5,
 'estimators': [('SGD_Classifier', SGDClassifier(loss='log_loss')),
  ('XGB_Classifier',
   XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 gamma=None, grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=6.976932788820359,
                 max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
                 max_delta_step=None, max_depth=None, max_leaves=None,
                 min_child_weight=None, missing=nan, monotone_constraints=None,
                 multi_strategy=None, n_estimators=872, n_jobs=None,
                 num_parallel_tree=None, random_state=None, ...)),
  ('Random Forest',
   RandomForestClassifier(class_weight='balanced_subsample', criterion='

In [475]:
s_clf_run.data.params

{'estimators': "[('SGD_Classifier', SGDClassifier()), ('Random Forest', RandomForestClassifier()), ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier()), ('Bagging_Classifier', BaggingClassifier()), ('Ada_Boost', AdaBoostClassifier()), ('Grad_Boost', GradientBoostingClassifier())]",
 'final_estimator': "SGDClassifier(loss='log_loss')",
 'cv': '5',
 'stack_method': 'auto',
 'passthrough': 'False',
 'SGD_Classifier_Params': "{'alpha': 0.0001, 'average': False, 'early_stopping': False, 'epsilon': 0.1, 'eta0': 0.0, 'fit_intercept': True, 'l1_ratio': 0.15, 'learning_rate': 'optimal', 'loss': 'log_loss', 'max_iter': 1000, 'n_iter_no_change': 5, 'penalty': 'l2', 'power_t': 0.5, 'shuffle': True, 'tol': 0.001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}",
 'XGB_Classifier_Params': "{'objective': 'binary:logistic', 'booster': 'gbtree', 'enable_categorical': False, 'learning_rate': 6.976932788820359, 'missing': nan, 'n_estimators': 872, 'tree_method': 'hist'}",
 'Ran

In [478]:
best_model1 = mlflow.sklearn.load_model('mlflow-artifacts:/11856c6b7da848c38f85aa3ece46ad6c/5c209d92a1174a5aaefc84258e073304/artifacts/optuna_Stacked_Classifier')
best_model1.get_params()

[2024-01-25 16:13:40,354: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow-artifacts/artifacts/11856c6b7da848c38f85aa3ece46ad6c/5c209d92a1174a5aaefc84258e073304/artifacts/optuna_Stacked_Classifier/requirements.txt]


{'cv': 5,
 'estimators': [('SGD_Classifier', SGDClassifier(loss='log_loss')),
  ('XGB_Classifier',
   XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 gamma=None, grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=6.976932788820359,
                 max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
                 max_delta_step=None, max_depth=None, max_leaves=None,
                 min_child_weight=None, missing=nan, monotone_constraints=None,
                 multi_strategy=None, n_estimators=872, n_jobs=None,
                 num_parallel_tree=None, random_state=None, ...)),
  ('Random Forest',
   RandomForestClassifier(class_weight='balanced_subsample', criterion='

In [482]:
import mlflow.xgboost

In [7]:
xgboost = mlflow.xgboost.load_model('mlflow-artifacts:/1cd799950ea74cae9dbec616b6b65993/acc08000975c49d7a3818110253855f1/artifacts/optuna_XGB_Classifier')

In [9]:
params = client.get_run('acc08000975c49d7a3818110253855f1').data.params

In [10]:
params

{'objective': 'binary:logistic',
 'base_score': 'None',
 'booster': 'gbtree',
 'callbacks': 'None',
 'colsample_bylevel': 'None',
 'colsample_bynode': 'None',
 'colsample_bytree': 'None',
 'device': 'None',
 'early_stopping_rounds': 'None',
 'enable_categorical': 'False',
 'eval_metric': 'None',
 'feature_types': 'None',
 'gamma': 'None',
 'grow_policy': 'None',
 'importance_type': 'None',
 'interaction_constraints': 'None',
 'learning_rate': '3.6523285164797543',
 'max_bin': 'None',
 'max_cat_threshold': 'None',
 'max_cat_to_onehot': 'None',
 'max_delta_step': 'None',
 'max_depth': 'None',
 'max_leaves': 'None',
 'min_child_weight': 'None',
 'missing': 'nan',
 'monotone_constraints': 'None',
 'multi_strategy': 'None',
 'n_estimators': '829',
 'n_jobs': 'None',
 'num_parallel_tree': 'None',
 'random_state': 'None',
 'reg_alpha': 'None',
 'reg_lambda': 'None',
 'sampling_method': 'None',
 'scale_pos_weight': 'None',
 'subsample': 'None',
 'tree_method': 'hist',
 'validate_parameters': '

In [11]:
with mlflow.start_run(experiment_id='188'):
    mlflow.log_params(params)

In [12]:
params_ = client.get_run('13ba590fba3f4584ba34cf81006dc31d').data.params

In [13]:
params_

{'objective': 'binary:logistic',
 'base_score': 'None',
 'booster': 'gbtree',
 'callbacks': 'None',
 'colsample_bylevel': 'None',
 'colsample_bynode': 'None',
 'colsample_bytree': 'None',
 'device': 'None',
 'early_stopping_rounds': 'None',
 'enable_categorical': 'False',
 'eval_metric': 'None',
 'feature_types': 'None',
 'gamma': 'None',
 'grow_policy': 'None',
 'importance_type': 'None',
 'interaction_constraints': 'None',
 'learning_rate': '3.6523285164797543',
 'max_bin': 'None',
 'max_cat_threshold': 'None',
 'max_cat_to_onehot': 'None',
 'max_delta_step': 'None',
 'max_depth': 'None',
 'max_leaves': 'None',
 'min_child_weight': 'None',
 'missing': 'nan',
 'monotone_constraints': 'None',
 'multi_strategy': 'None',
 'n_estimators': '829',
 'n_jobs': 'None',
 'num_parallel_tree': 'None',
 'random_state': 'None',
 'reg_alpha': 'None',
 'reg_lambda': 'None',
 'sampling_method': 'None',
 'scale_pos_weight': 'None',
 'subsample': 'None',
 'tree_method': 'hist',
 'validate_parameters': '

In [15]:
rand_f_model = mlflow.sklearn.load_model("mlflow-artifacts:/489d8cb8991a4e58b57565bc00e31b9e/34cc0da17feb4e0ea750551a65809ba8/artifacts/optuna_Random Forest")

[2024-01-25 18:38:57,171: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow-artifacts/artifacts?path=489d8cb8991a4e58b57565bc00e31b9e%2F34cc0da17feb4e0ea750551a65809ba8%2Fartifacts%2Foptuna_Random+Forest]


[2024-01-25 18:39:17,565: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow-artifacts/artifacts/489d8cb8991a4e58b57565bc00e31b9e/34cc0da17feb4e0ea750551a65809ba8/artifacts/optuna_Random%20Forest/MLmodel]
[2024-01-25 18:39:17,581: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow-artifacts/artifacts/489d8cb8991a4e58b57565bc00e31b9e/34cc0da17feb4e0ea750551a65809ba8/artifacts/optuna_Random%20Forest/model.pkl]
[2024-01-25 18:39:17,583: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end c

In [17]:
rand_f_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced_subsample',
 'criterion': 'log_loss',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 124,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [18]:
rand_f_params = client.get_run('34cc0da17feb4e0ea750551a65809ba8').data.params
rand_f_params

{'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'class_weight': 'balanced_subsample',
 'criterion': 'log_loss',
 'max_depth': 'None',
 'max_features': 'log2',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '124',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': 'None',
 'verbose': '0',
 'warm_start': 'False'}

In [19]:
with mlflow.start_run(experiment_id='181'):
    mlflow.log_params(rand_f_params)